In [1]:
# !pip install torch
import numpy as np
import random
import json
import nltk
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
stemmer = PorterStemmer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kmgav\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

In [3]:
def stem(word):
    return stemmer.stem(word.lower())

In [4]:
def bag_of_words(tokenized_sentence, words):
    sentence_words= [stem(word) for word in tokenized_sentence]
    bag= np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words: 
            bag[idx]= 1
    return bag

In [5]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1= nn.Linear(input_size, hidden_size) 
        self.l2= nn.Linear(hidden_size, hidden_size) 
        self.l3= nn.Linear(hidden_size, num_classes)
        self.relu= nn.ReLU()

    def forward(self, x):
        out= self.l1(x)
        out= self.relu(out)
        out= self.l2(out)
        out= self.relu(out)
        out= self.l3(out)
        return out

In [6]:
with open('intents.json', 'r') as f:
    intents= json.load(f)

In [7]:
all_words= []
tags= []
xy= []
for intent in intents['intents']:
    tag= intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        w= tokenize(pattern)
        all_words.extend(w)
        xy.append((w, tag))

In [8]:
ignore_words= ['?', '.', '!']
all_words= [stem(w) for w in all_words if w not in ignore_words]
all_words= sorted(set(all_words))
tags= sorted(set(tags))

print(len(xy), "patterns")
print(len(tags), "tags:", tags)
print(len(all_words), "unique stemmed words:", all_words)

33 patterns
10 tags: ['appointment', 'goodbye', 'greeting', 'hours', 'mopeds', 'opentoday', 'payments', 'rental', 'thanks', 'today']
54 unique stemmed words: ["'d", "'s", 'a', 'accept', 'anyon', 'appoint', 'are', 'bye', 'can', 'card', 'cash', 'credit', 'day', 'do', 'doe', 'for', 'good', 'goodby', 'have', 'hello', 'help', 'hi', 'hour', 'how', 'i', 'is', 'kind', 'later', 'like', 'mastercard', 'mope', 'need', 'of', 'onli', 'open', 'rent', 'see', 'take', 'tata', 'thank', 'that', 'the', 'there', 'thi', 'to', 'today', 'want', 'we', 'what', 'when', 'which', 'work', 'you', 'your']


In [9]:
#Train data
X_train= []
y_train= []
for (pattern_sentence, tag) in xy:
    bag= bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    label= tags.index(tag)
    y_train.append(label)

X_train= np.array(X_train)
y_train= np.array(y_train)

In [10]:
#parameter tuning
num_epochs= 1000
batch_size= 8
learning_rate= 0.001
input_size= len(X_train[0])
hidden_size= 8
output_size= len(tags)
print(input_size, output_size)

54 10


In [11]:
class ChatDataset(Dataset):
    def __init__(self):
        self.n_samples= len(X_train)
        self.x_data= X_train
        self.y_data= y_train

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.n_samples

In [12]:
dataset= ChatDataset()
train_loader= DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

In [13]:
device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [14]:
model= NeuralNet(input_size, hidden_size, output_size).to(device)

In [15]:
criterion= nn.CrossEntropyLoss()
optimizer= torch.optim.Adam(model.parameters(), lr=learning_rate)

In [16]:
#Model training
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words= words.to(device)
        labels= labels.to(dtype=torch.long).to(device)
        outputs= model(words)
        loss= criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1)%100== 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
print(f'final loss: {loss.item():.4f}')

Epoch [100/1000], Loss: 1.9724
Epoch [200/1000], Loss: 0.4803
Epoch [300/1000], Loss: 0.2350
Epoch [400/1000], Loss: 0.0033
Epoch [500/1000], Loss: 0.0163
Epoch [600/1000], Loss: 0.0018
Epoch [700/1000], Loss: 0.0025
Epoch [800/1000], Loss: 0.0001
Epoch [900/1000], Loss: 0.0029
Epoch [1000/1000], Loss: 0.0001
final loss: 0.0001


In [17]:
data= {
"model_state": model.state_dict(),
"input_size": input_size,
"hidden_size": hidden_size,
"output_size": output_size,
"all_words": all_words,
"tags": tags
}

In [18]:
FILE= "data.pth"
torch.save(data, FILE)
print(f'Training done. file saved to {FILE}')

Training done. file saved to data.pth


In [19]:
with open('intents.json', 'r') as json_data:
    intents= json.load(json_data)
FILE= "data.pth"
data= torch.load(FILE)

In [20]:
input_size= data["input_size"]
hidden_size= data["hidden_size"]
output_size= data["output_size"]
all_words= data['all_words']
tags= data['tags']
model_state= data["model_state"]

model= NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

NeuralNet(
  (l1): Linear(in_features=54, out_features=8, bias=True)
  (l2): Linear(in_features=8, out_features=8, bias=True)
  (l3): Linear(in_features=8, out_features=10, bias=True)
  (relu): ReLU()
)

In [21]:
bot= "Doctor AI"
print("Let's chat! (type 'quit' to exit)")
while True:
    sentence= input("You: ")
    if sentence== "quit":
        break

    sentence= tokenize(sentence)
    X= bag_of_words(sentence, all_words)
    X= X.reshape(1, X.shape[0])
    X= torch.from_numpy(X).to(device)

    output= model(X)
    _,predicted= torch.max(output, dim=1)
    tag= tags[predicted.item()]

    probs= torch.softmax(output, dim=1)
    prob= probs[0][predicted.item()]
    if prob.item() > 0.75:
        for intent in intents['intents']:
            if tag== intent["tag"]:
                print(f"{bot}: {random.choice(intent['responses'])}")
    else:
        print(f"{bot}: I do not understand, say again please..")

Let's chat! (type 'quit' to exit)
You: Hello
Doctor AI: Good to see you again
You: open today
Doctor AI: We're open every day from 9am-9pm
You: appointment needed
Doctor AI: Yeah, I m there for you i ll book
You: do you have mopeds
Doctor AI: We have Piaggio, Vespa and Yamaha mopeds
You: i need moped for rent
Doctor AI: We rent Yamaha, Piaggio and Vespa mopeds
You: will you accept credit cards 
Doctor AI: We accept most major credit cards
You: thanks
Doctor AI: My pleasure
You: bye
Doctor AI: Have a nice day
You: quit
